In [1]:
# libraries
import numpy as np
import pandas as pd 
import torch
from transformers import XLNetConfig, XLNetForTokenClassification, TrainingArguments
import random
from torch.nn.utils.rnn import pad_sequence
from torchmetrics.functional import pearson_corrcoef
from torchmetrics import Metric
from ipynb.fs.full.utils_dh import RegressionTrainerMAE, RegressionTrainerPCC, RegressionTrainerTwoMAEPCC, RegressionTrainerTwoNormMAE, RiboDatasetGWS, GWSDatasetFromPandas, CorrCoef, collate_fn, compute_metrics, compute_metrics_saved  # custom dataset and trainer

/nfs_home/nallapar/miniconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-09 15:57:53.895482: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-09 15:57:56.241747: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# model parameters
annot_thresh = 0.5
longZerosThresh_val = 20
percNansThresh_val = 0.05
d_model_val = 256
n_layers_val = 3
n_heads_val = 8
dropout_val = 0.3
lr_val = 1e-4
batch_size_val = 1
loss_fun_name = '2LNormMAE' # either 3LMAE or 3LPCC or 2LDPCCCMAE or 2LNormMAE
alpha = 1 # alpha value for the deprivation difference loss term

In [3]:
# reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# dataset paths 
data_folder = '/net/lts2gdk0/mnt/scratch/lts2/nallapar/rb-prof/data/Darnell_Full/Darnell/data_conds_split/processed/'

# model name and output folder path
model_name = 'XLNetDepr-' + str(n_layers_val) + '_' + str(d_model_val) + '_' + str(n_heads_val) + '-ALL_NA-PEL-BS' + str(batch_size_val) + '-GWS_PCC_IT192_DH_' + str(loss_fun_name) + 'ALPHA' + str(alpha) + '_NZ' + str(longZerosThresh_val) + '_PNTh' + str(percNansThresh_val) + '_AnnotThresh' + str(annot_thresh)
output_loc = "saved_models/" + model_name

In [4]:
# generate dataset
ds = 'ALL' # uses all the three conditions
train_dataset, test_dataset = RiboDatasetGWS(data_folder, ds, threshold = annot_thresh, longZerosThresh = longZerosThresh_val, percNansThresh = percNansThresh_val)

# convert pandas dataframes into torch datasets
train_dataset = GWSDatasetFromPandas(train_dataset)
test_dataset = GWSDatasetFromPandas(test_dataset)
print("samples in train dataset: ", len(train_dataset))
print("samples in test dataset: ", len(test_dataset))

samples in train dataset:  6386
samples in test dataset:  1592


In [5]:
# load xlnet to train from scratch
config = XLNetConfig(vocab_size=193, pad_token_id=192, d_model = d_model_val, n_layer = n_layers_val, n_head = n_heads_val, d_inner = d_model_val, num_labels = 1, dropout=dropout_val) # 5^3 + 1 for padding
model = XLNetForTokenClassification(config)

# modify the output layer
model.classifier = torch.nn.Linear(d_model_val, 2, bias=True)

In [6]:
# xlnet training arguments
training_args = TrainingArguments(
    output_dir = output_loc,
    learning_rate = lr_val,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = batch_size_val, # training batch size = per_device_train_batch_size * gradient_accumulation_steps
    per_device_eval_batch_size = 1,
    eval_accumulation_steps = 4, 
    num_train_epochs = 200,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    push_to_hub = False,
    dataloader_pin_memory = True,
    save_total_limit = 5,
    dataloader_num_workers = 4,
    include_inputs_for_metrics = True
)

# initialize trainer
if loss_fun_name == '3LMAE':
    trainer = RegressionTrainerMAE(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        data_collator=collate_fn,
        compute_metrics=compute_metrics,
        alpha = alpha
    )
elif loss_fun_name == '3LPCC':
    trainer = RegressionTrainerPCC(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        data_collator=collate_fn,
        compute_metrics=compute_metrics,
        alpha = alpha
    )
elif loss_fun_name == '2LDPCCCMAE':
    trainer = RegressionTrainerTwoMAEPCC(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        data_collator=collate_fn,
        compute_metrics=compute_metrics,
        alpha = alpha
    )
elif loss_fun_name == '2LNormMAE':
    trainer = RegressionTrainerTwoNormMAE(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        data_collator=collate_fn,
        compute_metrics=compute_metrics,
        alpha = alpha
    )

In [7]:
# train model
trainer.train()

# save best model
trainer.save_model(output_loc + "/best_model")

/nfs_home/nallapar/miniconda3/envs/tf/lib/python3.9/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/nfs_home/nallapar/miniconda3/envs/tf/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged

/nfs_home/nallapar/miniconda3/envs/tf/lib/python3.9/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


tensor(2.2003, device='cuda:0', grad_fn=<DivBackward0>) tensor(-19.4177, device='cuda:0', grad_fn=<DivBackward0>)


Epoch,Training Loss,Validation Loss,R
1,0.000000,nan,nan
2,0.000000,nan,nan


tensor(2.0261, device='cuda:0', grad_fn=<DivBackward0>) tensor(inf, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBa

/nfs_home/nallapar/miniconda3/envs/tf/lib/python3.9/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackw

/nfs_home/nallapar/miniconda3/envs/tf/lib/python3.9/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackward0>) tensor(nan, device='cuda:0', grad_fn=<DivBackward0>)
tensor(nan, device='cuda:0', grad_fn=<DivBackw

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # train model                                                                                │
│ ❱ 2 trainer.train()                                                                              │
│   3                                                                                              │
│   4 # save best model                                                                            │
│   5 trainer.save_model(output_loc + "/best_model")                                               │
│                                                                                                  │
│ /nfs_home/nallapar/miniconda3/envs/tf/lib/python3.9/site-packages/transformers/trainer.py:1664   │
│ in train                                                                                         │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /nfs_home/nallapar/miniconda3/envs/tf/lib/python3.9/site-packages/transformers/trainer.py:1940   │
│ in _inner_training_loop                                                                          │
│                                                                                                  │
│   1937 │   │   │   │   │   with model.no_sync():                                                 │
│   1938 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1939 │   │   │   │   else:                                                                     │
│ ❱ 1940 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1941 │   │   │   │                                                                             │
│   1942 │   │   │   │   if (                                                                      │
│   1943 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /nfs_home/nallapar/miniconda3/envs/tf/lib/python3.9/site-packages/transformers/trainer.py:2735   │
│ in training_step                                                                                 │
│                                                                                                  │
│   2732 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2733 │   │                                                                                     │
│   2734 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2735 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2736 │   │                                                                                     │
│   2737 │   │   if self.args.n_gpu > 1:                     

In [ ]:
# evaluate model
trainer.evaluate()